In [1]:
import xml.etree.ElementTree as ET
import re

# Charger le fichier SVG
def charger_svg(chemin_fichier):
    tree = ET.parse(chemin_fichier)
    root = tree.getroot()
    return tree, root

# Appliquer proprement une couleur sur un élément SVG
def appliquer_couleur(elem, couleur):
    style = elem.get('style')
    if style and 'fill:' in style:
        # Remplacer la valeur de fill dans style
        new_style = re.sub(r'fill:[^;]+', f'fill:{couleur}', style)
        elem.set('style', new_style)
    elif style:
        # Ajouter fill au style existant
        elem.set('style', style + f';fill:{couleur}')
    elif elem.get('fill'):
        # Modifier l'attribut fill directement
        elem.set('fill', couleur)
    else:
        # Ajouter un style avec fill uniquement
        elem.set('style', f'fill:{couleur}')

# Fonction principale pour changer la couleur d'un département donné
def changer_couleur_departement(fichier_entree, fichier_sortie, liste_departements, couleur):
    tree, root = charger_svg(fichier_entree)

    ET.register_namespace('', 'http://www.w3.org/2000/svg')
    ET.register_namespace('xlink', 'http://www.w3.org/1999/xlink')

    trouve = 0


    for departement in liste_departements:
        try:
            int(departement)
        except:
            if departement == "2A" or departement == "2B":
                pattern1 = re.compile("Dep" + str(departement))
                pattern2 = re.compile("Dep" + str(departement))
            else:
                print("Ce n'est pas un département")
                return
        else:
            if 1 <= int(departement) <= 19 or 21 <= int(departement) <= 74 or 76 <= int(departement) <= 91 or int(departement) == 95:
                pattern1 = re.compile("Dep" + str(departement))
                pattern2 = re.compile("Dep" + str(departement))
            elif int(departement) == 75 or 92 <= int(departement) <= 94:
                pattern1 = re.compile("Dep" + str(departement))
                pattern2 = re.compile("n0" + str(departement))
            elif 971 <= int(departement) <= 974 or int(departement) == 976:
                pattern1 = re.compile("n" + str(departement))
                pattern2 = re.compile("n" + str(departement))
            else:
                print("Ce n'est pas un département")
                return



        for elem in root.iter():
            elem_id = elem.get('id')
            if elem_id and (pattern1.match(elem_id) or pattern2.match(elem_id)):
                if elem.tag.endswith('g'):
                    # C’est un groupe, on applique la couleur à ses enfants
                    for child in elem:
                        appliquer_couleur(child, couleur)
                    trouve += 1
                else:
                    # Élément direct (comme <path>), on applique directement
                    appliquer_couleur(elem, couleur)
                    trouve += 1

    print(f"{trouve} élément(s) modifié(s).")
    tree.write(fichier_sortie, encoding='utf-8', xml_declaration=True)




def txt_to_liste(file_txt):
    file = open(file_txt, encoding= "utf-8")
    liste = []
    for ligne in file:
        if ligne[15:16] == ' ':
            liste.append(ligne[13:15])
        else:
            liste.append(ligne[13:16])
    return [dept for dept in liste if dept != ""]



def fusionner_html_svg(head_path, svg_path, foot_path, output_path):
    try:
        with open(head_path, 'r', encoding='utf-8') as f_head:
            head_content = f_head.read()

        with open(svg_path, 'r', encoding='utf-8') as f_svg:
            svg_content = f_svg.read()

        with open(foot_path, 'r', encoding='utf-8') as f_foot:
            foot_content = f_foot.read()

        with open(output_path, 'w', encoding='utf-8') as f_output:
            f_output.write(head_content)
            f_output.write(svg_content)
            f_output.write(foot_content)

        print(f"✅ Fichier combiné avec succès dans '{output_path}'")

    except Exception as e:
        print(f"❌ Erreur lors de la fusion : {e}")



## ⮟ - A EXECUTER - ⮟

In [2]:
vert_correct = "#85ed21"

changer_couleur_departement(
    fichier_entree = 'files/France_départementale.svg',
    fichier_sortie = 'files/dept_un_par_jour.svg',
    liste_departements = txt_to_liste('departements_du_jour.txt'),
    couleur=vert_correct
)

vert_correct = "#85ed21"
changer_couleur_departement(
    fichier_entree ='files/France_départementale.svg',
    fichier_sortie ='files/dept_six_par_jour.svg',
    liste_departements = txt_to_liste('6_departements_par_jour.txt'),
    couleur = vert_correct
)

fusionner_html_svg("files/haut_html_1.html", "files/dept_un_par_jour.svg", "files/bas_html.html", "dept_un_par_jour.html")
fusionner_html_svg("files/haut_html_6.html", "files/dept_six_par_jour.svg", "files/bas_html.html", "dept_six_par_jour.html")

40 élément(s) modifié(s).
106 élément(s) modifié(s).
✅ Fichier combiné avec succès dans 'dept_un_par_jour.html'
✅ Fichier combiné avec succès dans 'dept_six_par_jour.html'
